# Introduction

In intro-to-neural-nets, we learned what neural nets are and trained a simple one to recongize hand-written digits. In this notebook, we'll go into a little more depth into the math that powers the training. This notebook assumes you know a little calculus.

# Training Overview

Recall once we had our MNIS training data (input images + the associated digits), there were just a few lines of code to create and train a simple three-layer neural net that could recognize these digits with about 98% accuracy:

```python
neural_net = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),          # Input layer; takes in a 28x28 pixel image, flattens it.
    tf.keras.layers.Dense(64, activation='relu'),           # Hidden layer; 64 neurons, with ReLU activation
    tf.keras.layers.Dense(10, activation='softmax')         # Output layer; 10 neurons, with softmax activation
])
neural_net.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
neural_net.fit(train_images, train_labels, epochs=5)
```

There is a lot of math to unpack in how this works, so let's delve into it. We'll start with the first 4 lines (the neural net itself, which starts with random weights and so will make random predictions) before jumping into the last two lines (training it so it makes good predictions).

## Trainable parameters
Recall that each neuron outputs some linear combination of it's inputs. In particular:
* It takes n inputs: x<sub>1</sub>, x<sub>2</sub>, ..., x<sub>n</sub>
* It produces one output: x<sub>1</sub>\*w<sub>1</sub> + ... + x<sub>n</sub>\*w<sub>n</sub> + b
* It has n+1 trainable parameters: n weights (w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and a bias b
<center><img src="images/Artificial-neuron.png"/></center>

It's easier to use vector notation: X\*W + b, where X and W are n-dimensional vectors and \* is the dot-product.

In fact, it's even more efficient to use [matrix multiplication](https://en.wikipedia.org/wiki/Matrix_multiplication), where the matrix's columns are weight vectors for each neuron in that layer. GPUs can then compute the entire layers output in a single operation (rather than having a CPU do n*m multiplication operations).


## Activation functions
Each layer of neurons performs a linear function on it's input (matrix multiplication). But chaining together layers of linear functions is pointless because:
1. If f(X) and g(X) are linear functions, then so is f(g(X)).
2. So if a neural net has 2 inputs and 1 output and only does linear processing, then it doesn't matter how many hidden layers it has or how wide each layer is; even if it has 5 layers and a million parameters, it will be equivalent to a neural net consisting of just 1 neuron with 2 inputs and 3 parameters (that is; a function of the form f(x, y) = a\*x + b\*y + c)

To make neural nets capable of more than computing simple linear functions, we need to put something between the linear layers. That something is called an activation function.
<center><img src="images/neural-layer-with-activation.png"/></center>

Let's look at a few important activation functions:
<center><img src="images/activation_functions.png"/></center>

To the left is the human brain's activation function; it is a threshold/step function that outputs a fixed signal only if the weighted inputs are above a threshold. The step function doesn't work well for artificial neural nets because the training algorithm, which we'll talk about shortly, won't work if the derivative is zero everwhere.

The right three are the most commonly used activation functions in artificial neural nets:

* [**ReLU**](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)) - typical activation between layers; super simple and surprisingly effective.
* [**sigmoid**](https://en.wikipedia.org/wiki/Sigmoid_function) - typical output activation for binary classification networks (e.g, output is a single probability; true/false, cancer/not cancer, etc). It takes a number and outputs a number between 0 and 1. If the output is > 50%, it predicts positive. 
* [**softmax**](https://en.wikipedia.org/wiki/Softmax_function) - typical output activation for multi-class classifiers (e.g, which of the 10 digits is it). It takes n numbers and outputs n probabilities that sum is 1. The models prediction is the output with the highest probability.

OK; we are now in a position to understand the 4 lines of code that build our simple neural net. Run the following cell.

In [ ]:
import tensorflow as tf
neural_net = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28), name="input"),    # 28x28 2D input image => 784 1D output vector 
    tf.keras.layers.Dense(64, activation='relu', name="hidden"),    # 784 dim input => 64 dim output (matrix multiplication + ReLU activation)
    tf.keras.layers.Dense(10, activation='softmax', name="output")  # 64 dim => 10 propabilities (matrix multiplication + softmax activation)
])
neural_net.summary()


Some questions before we continue:
1. How many trainable parameters does the input layer have? So will it get better with training?
2. Why does the hidden layer have 50240 trainable parameters?
3. Why does the output layer have 650 trainable parameters?
4. Why was the ReLU activation function used in the hidden layer (vs no activation function)?
5. Why was the softmax activation function used in the output layer?

## Loss function

We've build a neural net, but initially the weights are all random, so we next need to train it. Training a neural net consists of iteratively performing two steps:
1. A **forward pass**: Giving the network a batch of training input, and seeing how close it's predicted output is to the expected output.
2. A **backwards pass**: Adjusting the network parameters slightly so the next forward pass will be closer

In order to perform the first step, we need a function to measure how close the models predictions are from the expected value. That is called a *loss function*.
* It takes two inputs; the expected and predicted values
* It outputs one value; a number indicating how close the two are (0 = a perfect match)

Common loss functions include:
* [**Mean square error**](https://en.wikipedia.org/wiki/Mean_squared_error) - used for regression models (models that output a number), like predicting the price of a home.
* [**Cross entropy (aka log loss)**](https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a) - used for classification models (models that output probabilities; typically using sigmoid or softmax output activations), such as cancer/not cancer, which digit was drawn, etc. You could use MSE for these too, but this function is better at heavily penalizing high probability predictions that are wrong (which will help during gradient descent, our next topic).

## Gradient descent

At this point, we've:
* Obtained some training data (inputs, expected outputs)
* Built a neural net with trainable parameters
* Defined a loss function to measure how far off the neural nets predictions are on the training data after performing a *forward pass* on a batch of training data

Next up is adjusting the parameters (training them) in order to minize the loss function (so that the model's predicted output is closer to the expected output). Of course we know from calculus the minimum occurs when the derivative is zero. But unlike the simple equations you get in a first year calculus class, the neural net equations are way too complicated to solve in closed form.

For each batch of training data we process in a forward pass, we can compute the derivative of the loss function with respect to each parameter (this is called a partial derivative) by holding the inputs and all other weights constant. This is possible because all the neural net functions we've used (linear transforms and activations) are differentiable. We'll talk more about how the derivative is computed later (it uses a cool technique called "back propogation" which is based on the chain rule), but for now assume we can compute it efficiently. We then adjust the weights slightly in the negative direction of the derivative. We do this again and again for multiple training batches and that process is called gradient descent. Each step we adjust the weight by the gradient times a small number called the learning rate. Here's what it might look like for one parameter:
<center><img src="images/one_parameter_gradient_descent.png"/></center>


Here's what it might look like for two parameters. 
<center><img src="images/two_parameter_gradient_descent.png"/></center>
Many people ask; can this get stuck in a local minima? In theory, yes. But in practice, no. That's because in higher dimension spaces (our simple network has 12k dimensions, chatGPT has hundreds of billions) we don't see local minima in practice - just saddle points.

The problem with a vanilla gradient descent optimizer is that if the training data is large and you have to processes in batches, it can bounce around too much; it's *stochastic* as each batch of training data moves it in a slightly different direction, and can sometimes overshoot the minima and have to come back if the learning rate is too high (and take forever to train if the learning rate is too low). Optimizers like **adam** build on sgd and add techniques like "momentum" (averaging the newly computed parameter updates with the previous ones, so things go in a more straight line):
<center><img src="images/stochastic_gradient_descent.png"/></center>

With that in mind, here's the updated code; hopefully you can now understand the last two lines:
* The optimizer is Adam - an optimized gradient descent algorithm
* The loss function is cross-entropy - measuring the expected vs predicted outputs for categorical predictions (in this case, 10 probabilities)
* And it trains over all the training data 5 times (5 epochs)

```python
neural_net = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),          # Input layer
    tf.keras.layers.Dense(64, activation='relu'),           # Hidden layer with ReLU activation
    tf.keras.layers.Dense(10, activation='softmax')         # Output layer with softmax activation
])
neural_net.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
neural_net.fit(train_images, train_labels, epochs=5)
```

As you learned in the intro notebook, this simple network gives over 97% accuracy.

# You try it

Frameworks like Keras and PyTorch perform "auto-grad"; automatically computing gradients during the forward pass on the training data so that they can adjust the weights for you during the backwards pass. We didn't explicitly specify a learning rate, so got the default. And the fact that gradients were computed and applied were all opaque to us as the framework took care of it.

The nice thing about Google's Keras is that it is easier to use for begginers and it was the first one created. However PyTorch is a newer framework by Meta that has started to become dominant because it is more flexible and easier to see under the covers. In the cells below, we'll use PyTorch to see the match behind the training.

The network below is super-simple because I just want to show the math in action:
1) One neuron; 2 inputs, 1 output
2) No activation function
3) Training data to teach it to compute f(x1, x2) = 2x1 + x2 + 1
Since it's a regression model, we'll use MSE loss.

Not might be a good time to talk about how backprop works.
* Our simple neural net has 3 parameters, w1, w2 and b, and computes f(x1, x2) = w1*x1 + w2*x2 + b
* We are using MSE loss; l(x, y) = (x-y)<sup>2</sup>

So after passing in training data X=(x1,x2) and expecting output y, then in order to figure out how to adjust the parameter w1 (for example), we need to compute *d/dw1* l(f(X), y). Let's see how that is done.

Recall during the forward pass, the data went through two steps (in a more complex network there would be more steps):
1) X -> f(X)           - the linear layer
2) f(X) -> l(f(X), y)  - the loss function

As this happened, the framework kept track of the computational graph, as well as the input and output at each step.

To perform backprop and adjusts the weights:
1. By the chain rule: *d/dw1* l(f(X), y) = *dl/dw1*(f(X), y) *df/dw1*(X).
2. But *dl/dw1*(f(X), y) is just 2(f(X)), since it's MSE. And the framework kept track of the the input f(X)=w1*x1 + w2*x2 + b during the forward pass, so it doesn't need to be recomputed.
3. *df/dw1*(X) is also easy to calculate. It's just x1! And again the framework kept track of x1 during the forward pass.
4. So *d/dw1* l(f(X), y) is just the products of the partial derivatives computed in the previous two steps. And since w1 is defined in this layer, it's value is updated by it's gradient times a small learning rate.

In summary; the derivative of the last function (the loss function) is computed first, then iteratively the derivative at each point in the computational graph is backproped so it can be multiplied by the derivative of the function at the previous layer. Along the way, weights are adjusted at each node in the graph.


OK; let's see this in action...

## But first, clone this notebook in your Google Colab account
Sign in to [Google colab](https://colab.research.google.com) and from there:
1. Go to "Open Notebook" (it should have popped up by default, but if not you can find it under the "File" menu)
2. Select "GitHub"
3. Enter the following path: https://github.com/marcshepard/teaching-data-science/blob/main/neural-net-math-overview.ipynb
4. Click the search icon

In [ ]:
# First let's create a model, some training data, and a few support functions.
# This is a bit verbose since it uses PyTorch, so don't worry too much about the details...

!pip install torchviz
import torch
from torch import nn
from torch import optim
from torchviz import make_dot
from IPython.display import Image

# Generate some training data; f(x1, x2) = 2*x1 + x2 + 1
x_train = torch.randn(1000, 2)                      # Inputs: 1000 pairs of random numbers
y_train = (2*x_train[:, 0] + x_train[:, 1] + 1)     # Outputs: y = 2*x1 + x2 + 1

# Define the model, optimizer, and loss function
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear = nn.Linear(2, 1)   # 1 neuron with 2 inputs

    def forward(self, x):
        x = self.linear(x)
        return x
model = Model()
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

# Function to show the models parameters - there should be 3: 2 weights and 1 bias
def print_parameters():
    print("Model parameters:")
    for name, param in model.named_parameters():
        print(name, param.data)

# A function to evaluate the model's loss when run on the training data
def evaluate_loss():
    model.eval()
    with torch.no_grad():
        outputs = model(x_train)
        loss = loss_fn(outputs.view(-1), y_train)
    return loss.item()

# A function to train the model for some number of epochs
def train(epochs):
    for _ in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(x_train)
        loss = loss_fn(outputs, y_train.view(-1, 1))
        loss.backward()
        optimizer.step()


In [ ]:
# Next, let's look at what happens to the parameters before and after 1 training epoch and visualize the computation graph

print ("We're training a 1 neural network with 2 inputs to approximate the function f(x1, x2) = 2*x1 + x2 + 1")
print ("We expect it to converge to weights of 2 and 1 and a bias of 1 after enough training epochs")

print ("Before training:")
print_parameters()
print ("\nMSE loss on the training data:", evaluate_loss())

print()
print("After training for 1 epoch:")
train(1)
print_parameters()
print ("\nMSE loss on the training data:", evaluate_loss())

# Let's visualize the computation graph associated with the loss
loss = loss_fn(model(x_train), y_train.view(-1, 1))
make_dot(loss, params=dict(model.named_parameters())).render("graph", format="png")
Image(filename='graph.png')

In [ ]:
# Now you try it a few times. Watch things get better as you increase the number of epochs!

epochs = 100 # TODO, change this as you see fit, or just run this cell multiple times to get it to converge

print("After training for {epochs} epoch:")
train(epochs)
print_parameters()
print ("\nMSE loss on the training data:", evaluate_loss())

# To learn more

If you are interested in learning more about the mathematics behind neural network training, Andrej Karpathy's series [Neural Networks: Zero to Hero](https://www.youtube.com/playlist?list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ) is an excellent resource.